In [ ]:
# import
import mlflow
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [ ]:
# Azure Machine Learning接続
## MLFLOW_TRACKING_URI環境変数にURIがセットされている場合はURIセット部分のみ省略可
subscription_id = "<subscription_id>"
resource_group = "<resource_group_name>"
workspace = "<workspace_name>"

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

azureml_mlflow_uri = ml_client.workspaces.get(
    ml_client.workspace_name
).mlflow_tracking_uri

mlflow.set_tracking_uri(azureml_mlflow_uri)

In [ ]:
# データをpandas データフレームとして読み込み
credit_df = pd.read_excel(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls",
    header=1,
    index_col=0
)

# 分割
_df, test_df = train_test_split(
    credit_df,
    test_size=0.2,
)

train_df, valid_df = train_test_split(
    _df,
    test_size=0.2,
)

# 加工
y_train = train_df.pop("default payment next month")
X_train = train_df.values
train_dataset = lgb.Dataset(X_train, label=y_train)

y_valid = valid_df.pop("default payment next month")
X_valid = valid_df.values
valid_dataset = lgb.Dataset(X_valid, label=y_valid)

y_test = test_df.pop("default payment next month")
X_test = test_df.values

In [ ]:
# LightGBMパラメーター設定
params = {
    "objective": "binary",
    "boosting_type": "gbdt",
    "num_leaves": 20,
    "max_depth": 10,
    "learning_rate": 0.1,
    "device_type": "cpu",
    "seed": 42,
    "deterministic": True,
}

In [ ]:
# 学習と実験管理
with mlflow.start_run() as run:  
    mlflow.log_params(params)
    
    # LightGBM学習
    clf = lgb.train(
        params,
        train_set=train_dataset,
        valid_sets=[valid_dataset],
        valid_names=['valid'],
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)]
    )
    artifact_path="DefaultPredictModel"
    model = mlflow.lightgbm.log_model(lgb_model=clf, artifact_path=artifact_path)

In [ ]:
# モデル登録
mlflow.register_model(model.model_uri, "DefaultPredictModel")

In [ ]:
# モデルを特定するURI
print(f"runs:/{run.info.run_id}/{artifact_path}")
print(model.model_uri)

In [ ]:
# モデル読み込み
loaded_model = mlflow.pyfunc.load_model(model_uri=model.model_uri)
loaded_model = mlflow.pyfunc.load_model(model_uri=f"runs:/{run.info.run_id}/{artifact_path}")
loaded_model = mlflow.pyfunc.load_model(model_uri="models:/DefaultPredictModel/latest")

In [ ]:
# 推論
loaded_model.predict(X_test)

In [ ]:
# 学習と実験管理 (自動ロギング)
with mlflow.start_run() as run:
    # 自動ロギング
    mlflow.lightgbm.autolog()

    clf = lgb.train(
        params,
        train_set=train_dataset,
        valid_sets=[valid_dataset],
        valid_names=['valid'],
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)]
    )